In [8]:
from dotenv import load_dotenv
from environment.game import Game
from environment.generators.random_grid_generator import RandomGridGenerator
from environment.grid_visualizer import GridVisualizer
from environment.types import Direction

load_dotenv(dotenv_path="./llm-dungeon-adventures/.env")

True

In [11]:
DUNGEON_SIZE, generator_seed = 50, 100

generator = RandomGridGenerator(DUNGEON_SIZE, generator_seed)
atmosphere = ['magic', 'sunny']

In [15]:
game = Game(generator, atmosphere)
text = "go north\ngo east\ngo east"
game.play(text)


TypeError: Game.__init__() takes 2 positional arguments but 3 were given